In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
train_data_dir = 'dataset/train'
validation_data_dir = 'dataset/valid'
test_data_dir = 'dataset/test'

# ImageDataGenerator for data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training data
train = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(299, 299),  # Resize images to match InceptionV3 input size
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

valid_test_datagen = ImageDataGenerator(rescale=1 / 255.0)

# Load validation data
validation = valid_test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(299, 299), # Resize images to match InceptionV3 input size
    batch_size=32,
    class_mode='categorical'
)

# Load test data
test = valid_test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(299, 299), # Resize images to match InceptionV3 input size
    batch_size=32,
    class_mode='categorical'
)

Found 7624 images belonging to 53 classes.
Found 265 images belonging to 53 classes.
Found 265 images belonging to 53 classes.


In [2]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping, History
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(53, activation='softmax')(x)  # Output layer for 53 classes

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of InceptionV3
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


model.summary() 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   96         ['conv2d[0][0]']                 
 alization)                     32)                                                           

In [3]:
# fix duplicate library error
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [4]:
callbacks = [EarlyStopping(monitor='loss', patience=3), History()]
history = model.fit(
        train,
        steps_per_epoch=len(train),
        epochs=200,
        validation_data=validation,
        validation_steps=len(validation), callbacks=callbacks
    )

Epoch 1/200
 55/239 [=====>........................] - ETA: 6:35 - loss: 3.6353 - accuracy: 0.1097

KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt
# Plot accuracy
plt.subplot(2,1,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

# Plot loss
plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
model.save("model.keras")

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('model.keras')

predictions = model.predict(test)

9/9 [==============================] - 14s 1s/step


In [ ]:
pred_classes = predictions.argmax(axis=-1)

print(pred_classes)
print(test.labels)


[42 23  2 28  2 28 42 14 45  0  5 11 15 29 13  0  9 46 24  9 36 43  7 23
  5 19 49 41  5 30 22  9  9 41 22 52 45 45 45  1 44 21 26 22  3 40 22 40
 30 14 38 13 35 29 22 27 47 16 35  2 35  9 45  5 41  4 28 43  4 49 47 13
  5 25 11  5 14 29 19 35 29  5 30  8  4 16 45 13 11 13 28  7 42 23 36 25
 30 26 21 52 16 41  3  8 22 42  5  6 36 13 22 20 14 35  1 51 28 11 45  6
 43 46 22 22 13 46 44 22 13 37 22 41 28 47 16 25 46  3 16 45 51 29  4 11
 33 48 44 42 35 24  2 45 26 18 23 10 22  1  3 22 42 20 14 52  5 46 43 30
 41  5 22 28 22 22 47 32 26 15 28  8  7 28 11 14 23 42 49  6  5 22 11 42
  8 37 21  0  8 38 38  4 49 40  8 29  0 11  5 14 24  5  8 41 48  3 18 30
  0 23 28 47 13 29  4 24  2 29 29 36 45 51 30  8 35 13 40  6 11 36 35 22
 13 42  1 12 29 26  2 22 45 43 11 13  4 11 28  2 47  2 22  3 32 14  3 41
 32]
[ 0  0  0  0  0  1  1  1  1  1  2  2  2  2  2  3  3  3  3  3  4  4  4  4
  4  5  5  5  5  5  6  6  6  6  6  7  7  7  7  7  8  8  8  8  8  9  9  9
  9  9 10 10 10 10 10 11 11 11 11 11 12 12 12 

In [ ]:
import numpy as np

def get_confusion_matrix(label, predictions, number_of_classes):
    confusion_matrix = np.zeros((number_of_classes, number_of_classes), dtype=int)
    
    for idx in range(len(label)):
        confusion_matrix[label[idx]][predictions[idx]] += 1
    
    return confusion_matrix

In [ ]:
confusion_matrix = get_confusion_matrix(test.labels, pred_classes, 53)

np.savetxt('confusion_matrix.txt', confusion_matrix, fmt='%i')

[[0 0 2 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
265


In [ ]:
def counts_from_confusion(confusion):
    """
    Obtain TP, FN FP, and TN for each class in the confusion matrix
    """

    counts_list = []

    # Iterate through classes and store the counts
    for i in range(confusion.shape[0]):
        tp = confusion[i, i]

        fn_mask = np.zeros(confusion.shape)
        fn_mask[i, :] = 1
        fn_mask[i, i] = 0
        fn = np.sum(np.multiply(confusion, fn_mask))

        fp_mask = np.zeros(confusion.shape)
        fp_mask[:, i] = 1
        fp_mask[i, i] = 0
        fp = np.sum(np.multiply(confusion, fp_mask))

        tn_mask = 1 - (fn_mask + fp_mask)
        tn_mask[i, i] = 0
        tn = np.sum(np.multiply(confusion, tn_mask))

        counts_list.append({'Class': i,
                            'TP': tp,
                            'FN': fn,
                            'FP': fp,
                            'TN': tn})

    return counts_list

In [ ]:
count_confusion = counts_from_confusion(confusion_matrix)

with open(r'count_confusion.txt', 'w') as fp:
    for cls in count_confusion:
        fp.write("%s\n" % cls)
    print('Done')

Done
